In [3]:
import pandas as pd

df = pd.read_csv("../data/winequality-white.csv", sep=";")
df.head(5)



,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [2]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")   # o http://127.0.0.1:5001 si cambiaste
mlflow.set_experiment("winequality-linear-regression")
print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://127.0.0.1:5000


In [4]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# ===== Carga de datos (ajusta a tu CSV) =====  # ejemplo del dataset de vino
target = "quality"
X = df.drop(columns=[target])
y = df[target]

num_cols = X.columns.tolist()

# ===== Dos variantes de preproceso para comparar =====
prep_std = ColumnTransformer(
    transformers=[("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                                    ("sc", StandardScaler())]), num_cols)],
    remainder="drop"
)

prep_mm = ColumnTransformer(
    transformers=[("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                                    ("sc", MinMaxScaler())]), num_cols)],
    remainder="drop"
)

pipe_std = Pipeline([("prep", prep_std), ("model", LinearRegression())])
pipe_mm  = Pipeline([("prep", prep_mm ), ("model", LinearRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def eval_and_log(name, pipe):
    with mlflow.start_run(run_name=name) as run:
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        mae  = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2   = r2_score(y_test, y_pred)

        mlflow.log_metrics({"MAE": mae, "RMSE": rmse, "R2": r2})
        mlflow.log_params({"model": "LinearRegression",
                           "prep": name})

        signature = infer_signature(X_train, pipe.predict(X_train))
        mlflow.sklearn.log_model(pipe, artifact_path="model", signature=signature,
                                 registered_model_name="winequality_lr")  # <- nombre en el registry

        print(name, {"MAE": mae, "RMSE": rmse, "R2": r2})
        return run.info.run_id

run_id_std = eval_and_log("std+linreg", pipe_std)
run_id_mm  = eval_and_log("minmax+linreg", pipe_mm)


c:\Users\galof\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/10 16:02:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\galof\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\tracking\_model_registry\utils.py:215: FutureWarning: Filesystem model registry backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri)
Registered model 'winequality_lr' already exists. Creating a new version of this model...
Created version '3' of model 'winequality_lr'.


std+linreg {'MAE': 0.5862665383250467, 'RMSE': np.float64(0.7543373063311434), 'R2': 0.2652750042179156}


2025/11/10 16:03:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


minmax+linreg {'MAE': 0.5862665383250468, 'RMSE': np.float64(0.7543373063311435), 'R2': 0.2652750042179155}


Registered model 'winequality_lr' already exists. Creating a new version of this model...
Created version '4' of model 'winequality_lr'.
